In [ ]:
!pip -qq install transformers

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nidhalyounes","key":"2a45ab973d7ad05ec340685be500b277"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [ ]:
!kaggle datasets download -d namanj27/ner-dataset
!unzip -q ner-dataset.zip

  0% 0.00/3.17M [00:00<?, ?B/s]
100% 3.17M/3.17M [00:00<00:00, 106MB/s]


In [ ]:
!pip -qq install barbar
!pip -qq install seqeval

     |████████████████████████████████| 51kB 6.9MB/s 


In [ ]:
from barbar import Bar
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch 
import transformers
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
data = pd.read_csv("/content/ner_datasetreference.csv", encoding="latin1").fillna(method="ffill")
data.tail(5)

,Sentence #,Word,POS,Tag
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O
1048574,Sentence: 47959,attack,NN,O


In [ ]:
from transformers import DistilBertTokenizer, DistilBertConfig,DistilBertForTokenClassification,AdamW

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from seqeval.metrics import f1_score, accuracy_score

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased', do_lower_case=False)

In [ ]:
MAX_LEN = 75
bs = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
tag_values = list(set(data["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [ ]:
haha= data["Tag"].values

In [ ]:
data["Tag"].unique()

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat'], dtype=object)

In [ ]:
dd= np.array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat','PAD'])

In [ ]:
list(set(dd))

['B-per',
 'I-tim',
 'B-gpe',
 'B-org',
 'I-nat',
 'I-org',
 'O',
 'I-geo',
 'B-art',
 'I-art',
 'I-gpe',
 'B-nat',
 'B-eve',
 'I-eve',
 'B-tim',
 'PAD',
 'I-per',
 'B-geo']

the begining



In [ ]:
def Datset_Ner(Data):
  Agg_F= Data.groupby("Sentence #")[["Word","Tag"]].agg(' '.join)
  Agg_F["tokens"]=""
  list_labels= []
  for i in trange(0,len(Agg_F)):
    for j in range(0,len(Agg_F["Word"][i].split(" "))):
      vv=Agg_F["Word"][i].split(" ")[j]
      jj= Agg_F["Tag"][i].split(" ")[j]
      ou= (vv,jj)
      list_labels.append(ou)
    Agg_F["tokens"][i]= list_labels
    list_labels=[]
  nbt= list(Agg_F.tokens)
  sentencx = [[word[0] for word in sentence] for sentence in nbt]
  lablx = [[word[1] for word in sentence] for sentence in nbt]
  return sentencx, lablx

In [ ]:
sentencx,lablx = Datset_Ner(data)

100%|██████████| 47959/47959 [00:26<00:00, 1782.22it/s]


In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []
    for word, label in zip(sentence, text_labels):
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)
        tokenized_sentence.extend(tokenized_word)
        labels.extend([label] * n_subwords)
    return tokenized_sentence, labels

In [ ]:
tokenized_texts_and_labels = [tokenize_and_preserve_labels(sent, labs)for sent, labs in zip(sentencx, lablx)]

In [ ]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
len(tokenized_texts)

47959

In [ ]:
len(labels)

47959

In [ ]:
for txt in tokenized_texts:
  print(txt)
  break

['Thousands', 'of', 'demons', '##tra', '##tors', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']


In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [ ]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [ ]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:
model= DistilBertForTokenClassification.from_pretrained("distilbert-base-cased",
    num_labels=18,
    output_attentions = False,
    output_hidden_states = False)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
                                {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                                 'weight_decay_rate': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                                 'weight_decay_rate': 0.0}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 3
max_grad_norm = 1.0

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:
model.cuda()

In [ ]:
def train_Ner(model, dataloader,optimizer, scheduler, n_expls):
  model= model.train()
  total_loss= 0
  for b_ix , d in enumerate(Bar(dataloader)):
    inputs_ids= d[0].cuda()
    attention_mask= d[1].cuda()
    targets= d[2].cuda()
    model.zero_grad()
    outputs= model(inputs_ids, attention_mask= attention_mask, labels= targets)
    loss = outputs[0]
    loss.backward()
    total_loss += loss.item()
    torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
    optimizer.step()
    scheduler.step()
  avg_train_loss = total_loss / n_expls
  return avg_train_loss

In [ ]:
def Val_Ner(model, dataloader,n_expls):
  model= model.eval()
  eval_loss, eval_accuracy = 0, 0
  loss_values, validation_loss_values = [], []
  predictions , true_labels = [], []
  with torch.no_grad():
    for idx, d in enumerate(Bar(dataloader)): 
      inputs_ids= d[0].cuda()
      attention_mask= d[1].cuda()
      targets= d[2].cuda()
      outputs= model(inputs_ids, attention_mask= attention_mask, labels= targets)
      logits = outputs[1].detach().cpu().numpy()
      targets = targets.to('cpu').numpy()
      eval_loss += outputs[0].mean().item()
      predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
      true_labels.extend(targets)
      eval_loss = eval_loss / n_expls
      validation_loss_values.append(eval_loss)
      pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
      valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
      accurancy= accuracy_score(pred_tags, valid_tags)
    return eval_loss ,  accurancy

In [ ]:
%%time
for epoch in range(epochs):
  print(f'Epoch {epoch + 1}/{epochs}')
  print('-' * 10)
  avg_train_loss= train_Ner(model, train_dataloader, optimizer, scheduler, len(train_dataloader))
  print("Train loss {}" .format(avg_train_loss))
  eval_loss_, accuracy = Val_Ner(model, valid_dataloader, len(valid_dataloader))
  print("Validation loss: {}".format(eval_loss_)) 
  print("Validation Accuracy: {}".format(accuracy))

Epoch 1/3
----------
43163/43163: [===============================>] - ETA 0.4s
Train loss 0.19457418027246237
4796/4796: [===============================>] - ETA 0.1s
Validation loss: 0.0007149586907612623
Validation Accuracy: 0.956770198610059
Epoch 2/3
----------
43163/43163: [===============================>] - ETA 0.2s
Train loss 0.11195799848375451
4796/4796: [===============================>] - ETA 0.1s
Validation loss: 0.0007012168551052192
Validation Accuracy: 0.9602615197127894
Epoch 3/3
----------
43163/43163: [===============================>] - ETA 0.2s
Train loss 0.08656565894720022
4796/4796: [===============================>] - ETA 0.1s
Validation loss: 0.0007569665937681896
Validation Accuracy: 0.9611508184842397
CPU times: user 10min 12s, sys: 6min 4s, total: 16min 16s
Wall time: 16min 21s


In [ ]:
torch.save(model.state_dict(), 'best_model_state.bin')

In [ ]:
test_sentence = """
Mr. Trump’s tweets began just moments after a Fox News report by Mike Tobin, a 
reporter for the network, about protests in Minnesota and elsewhere. 
"""


In [ ]:
model_2=DistilBertForTokenClassification.from_pretrained("distilbert-base-cased",
    num_labels=18,
    output_attentions = False,
    output_hidden_states = False)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

In [ ]:
model_2.load_state_dict(torch.load("/content/best_model_state.bin"))
model.eval()

In [ ]:
tokenized_sentence = tokenizer.encode(test_sentence)
input_ids = torch.tensor([tokenized_sentence])

In [ ]:
with torch.no_grad():
    output = model_2(input_ids)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

In [ ]:
# join bpe split tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(tag_values[label_idx])
        new_tokens.append(token)

In [ ]:
for token, label in zip(new_tokens, new_labels):
    print("{}\t{}".format(label, token))

O	[CLS]
B-per	Mr
B-per	.
I-per	Trump
O	’
O	s
O	tweets
O	began
O	just
O	moments
O	after
O	a
B-org	Fox
I-org	News
O	report
O	by
B-per	Mike
I-per	Tobin
O	,
O	a
O	reporter
O	for
O	the
O	network
O	,
O	about
O	protests
O	in
B-geo	Minnesota
O	and
O	elsewhere
O	.
O	[SEP]
